In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv("/content/amazon.csv")

In [6]:
df

,Text,label
0,This is the best apps acording to a bunch of ...,1
1,This is a pretty good version of the game for ...,1
2,this is a really . there are a bunch of levels...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
19991,this app is fricken stupid.it froze on the kin...,0
19992,Please add me!!!!! I need neighbors! Ginger101...,1
19993,love it! this game. is awesome. wish it had m...,1
19994,I love love love this app on my side of fashio...,1


In [7]:
df.shape

(19996, 2)

In [8]:
df["label"].value_counts()

,count
label,
1,15230
0,4766


In [9]:
df_class1 = df[df["label"] == 1]
df_class0 = df[df["label"]==0]

In [10]:
count_c1 ,count_c0 = df.label.value_counts()

In [11]:
count_c0

4766

In [12]:
count_c1

15230

In [13]:
df_class0.shape

(4766, 2)

In [14]:
df_class1.shape

(15230, 2)

In [15]:
df_class0.sample(count_c0)

,Text,label
14352,I tried to select a picture and I got to my pi...,0
2847,You can't do anything but shape clay on the wh...,0
10963,Not worth the free price. Will delete app. Th...,0
8779,"Downloaded it, tried to contact it to Facebook...",0
3182,why spend 2.99 if you could just look it up on...,0
...,...,...
14475,who represents a personal finance app with the...,0
18837,it is bad and I hate it if you get it dont if ...,0
16724,This is an absolutely ridiculous app! I'm 13 y...,0
13614,why would you need a note calender....? can't ...,0


In [16]:
df_oversample = df_class0.sample(count_c1,replace=True)

In [17]:
df_oversample

,Text,label
14668,Jump function simply doesn't work on my G2. W...,0
15427,Need to fix update. I cant even find the app t...,0
7976,it's not a real strategy game. all you do is p...,0
16714,"It's telling me I have a 50 year old brain, an...",0
11363,This game sucks. It was fun until I noticed it...,0
...,...,...
5022,This Ap loaded to my Kindle Fire however could...,0
8371,"If you are using an android , You simply don't...",0
7298,This app will not install on my droid x. When ...,0
2062,This download was the worst one yet. What on E...,0


In [18]:
new_df = pd.concat([df_class1,df_oversample],axis=0)

In [19]:
new_df["label"].value_counts()

,count
label,
1,15230
0,15230


In [20]:
new_df.shape

(30460, 2)

In [55]:
new_df["Text"][0]

'This is  the best apps acording to a bunch of people and I agree it has bombs eggs pigs TNT king pigs and realustic stuff'

In [21]:
new_df["label"].value_counts()

,count
label,
1,15230
0,15230


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
trainset,testset = train_test_split(new_df,test_size=0.2,random_state=42)

In [26]:
trainset.shape

(24368, 2)

In [27]:
testset.shape

(6092, 2)

In [28]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(trainset["Text"])
x_train = pad_sequences(tokenizer.texts_to_sequences(trainset["Text"]),maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(testset["Text"]),maxlen = 200)

In [29]:
print(x_train)

[[   0    0    0 ...  395  108    8]
 [   0    0    0 ...   78   67  123]
 [   0    0    0 ...  246   11  389]
 ...
 [   0    0    0 ...    7    7  114]
 [   0    0    0 ... 2287    3    9]
 [   0    0    0 ...    9  384   10]]


In [30]:
x_train.shape

(24368, 200)

In [31]:
print(x_test)

[[   0    0    0 ...   15   56   52]
 [   0    0    0 ...  353  434  279]
 [   0    0    0 ...    6  234  245]
 ...
 [   0    0    0 ...  277 4521  366]
 [   0    0    0 ...  104   33   26]
 [   0    0    0 ...   73   44  408]]


In [32]:
x_test.shape

(6092, 200)

In [33]:
y_train = trainset["label"]
y_test = testset["label"]

In [34]:
y_train.shape

(24368,)

In [35]:
y_test.shape

(6092,)

In [36]:
model =Sequential()

In [37]:
from tensorflow.keras.layers import Flatten

In [49]:
model.add(Embedding(input_dim = 5000,output_dim = 128,input_length = 200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))

In [50]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 1, 128)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 1, 128)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             258 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 2, 128)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,086,981 (11.78 MB)

 Trainable params: 3,086,981 (11.78 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [52]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['sequential/embedding/embeddings', 'sequential/lstm/lstm_cell/kernel', 'sequential/lstm/lstm_cell/recurrent_kernel', 'sequential/lstm/lstm_cell/bias', 'sequential/dense/kernel', 'sequential/dense/bias', 'sequential/embedding_1/embeddings', 'sequential/lstm_1/lstm_cell/kernel', 'sequential/lstm_1/lstm_cell/recurrent_kernel', 'sequential/lstm_1/lstm_cell/bias', 'sequential/dense_1/kernel', 'sequential/dense_1/bias', 'sequential/embedding_2/embeddings', 'sequential/lstm_2/lstm_cell/kernel', 'sequential/lstm_2/lstm_cell/recurrent_kernel', 'sequential/lstm_2/lstm_cell/bias', 'sequential/dense_2/kernel', 'sequential/dense_2/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


305/305 ━━━━━━━━━━━━━━━━━━━━ 65s 189ms/step - accuracy: 0.5041 - loss: 0.6935 - val_accuracy: 0.5146 - val_loss: 0.6928
Epoch 2/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 79s 180ms/step - accuracy: 0.5002 - loss: 0.6932 - val_accuracy: 0.5146 - val_loss: 0.6930
Epoch 3/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 83s 182ms/step - accuracy: 0.5091 - loss: 0.6931 - val_accuracy: 0.5146 - val_loss: 0.6930
Epoch 4/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 80s 177ms/step - accuracy: 0.4978 - loss: 0.6932 - val_accuracy: 0.4854 - val_loss: 0.6933
Epoch 5/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 87s 192ms/step - accuracy: 0.4921 - loss: 0.6932 - val_accuracy: 0.5146 - val_loss: 0.6931
Epoch 6/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 79s 181ms/step - accuracy: 0.5030 - loss: 0.6932 - val_accuracy: 0.4854 - val_loss: 0.6932
Epoch 7/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 86s 196ms/step - accuracy: 0.4944 - loss: 0.6932 - val_accuracy: 0.4854 - val_loss: 0.6938
Epoch 8/10
305/305 ━━━━━━━━━━━━━━━━━━━━ 77s 181ms/step - accuracy: 0.4965 - loss: 0.6933 - val

In [56]:
loss,accuracy = model.evaluate(x_test,y_test)
print("loss:",loss)
print("accuracy:",accuracy)

191/191 ━━━━━━━━━━━━━━━━━━━━ 23s 121ms/step - accuracy: 0.4934 - loss: 0.6932
loss: 0.6932001709938049
accuracy: 0.4924491047859192


In [57]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [58]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The sentiment of the review is: negative


In [59]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
The sentiment of the review is: negative


In [ ]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")